In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import tensorflow as tf # Work with tensors
from tensorflow import keras # Framework for DL
from tensorflow.keras import layers # Layers for NN
import missingno as msno # Tools for missing values
from PIL import Image # Image Tool
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.applications import VGG16

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os

import sys
import os
from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import adam, sgd, Adadelta 
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.applications import VGG16, VGG19
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
np.random.seed(1000)
import warnings
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D

from keras.callbacks import Callback, ModelCheckpoint
from keras.applications import VGG16,ResNet50,InceptionV3,MobileNet
from keras import optimizers
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras import layers
from keras import models
import tensorflow as tf
import numpy as np
import time
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications import VGG16
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam 
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D
from keras import callbacks

from keras import optimizers
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras import layers
from keras import models
import tensorflow as tf
import numpy as np
import time

import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import itertools


import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import tensorflow as tf

In [ ]:
IMAGE_LIB = '../input/finding-lungs-in-ct-data/2d_images/'
MASK_LIB = '../input/finding-lungs-in-ct-data/2d_masks/'


#IMAGE_LIB = '../input/cvcclinicdb/TIF/Original/'
#MASK_LIB = '../input/cvcclinicdb/TIF/Ground Truth/'

In [ ]:
"""
IMG_HEIGHT, IMG_WIDTH = 128, 128
SEED=42
epochs = 200
batchSize = 8
initial_learning_rate = 0.001
steps_per_epoch = 10

early_stop_patience = 50 #increase in the case that your model starts with very big learning rate
patience_reduce_lr=20
min_lr=1e-12
"""

"""






Epoch 00088: val_dice_coef did not improve from 0.55723
Epoch 89/200
10/10 [==============================] - 2s 226ms/step - loss: 0.4282 - precision_1: 0.9765 
- recall_1: 0.9977 - auc_1: 0.9988 - accuracy: 0.9939 - dice_coef: 0.5705 - jacard: 0.3994 
- val_loss: 0.4371 - val_precision_1: 0.9734 - val_recall_1: 0.9820 
- val_auc_1: 0.9918 - val_accuracy: 0.9900 - val_dice_coef: 0.5613 
- val_jacard: 0.3902








"""

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 128, 128
SEED=42
epochs = 20 #200
batchSize = 4
initial_learning_rate = 0.001
steps_per_epoch = 100

early_stop_patience = 80 #increase in the case that your model starts with very big learning rate
patience_reduce_lr=20
min_lr=1e-12

In [ ]:
all_images = [x for x in sorted(os.listdir(IMAGE_LIB)) if x[-4:] == '.tif']

x_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
for i, name in enumerate(all_images):
    im = cv2.imread(IMAGE_LIB + name, cv2.IMREAD_UNCHANGED).astype("int16").astype('float32')
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4)
    im = (im - np.min(im)) / (np.max(im) - np.min(im))
    x_data[i] = im

y_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
for i, name in enumerate(all_images):
    im = cv2.imread(MASK_LIB + name, cv2.IMREAD_UNCHANGED).astype('float32')/255.
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)
    y_data[i] = im

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (8,4))
ax[0].imshow(x_data[0], cmap='gray')
ax[1].imshow(y_data[0], cmap='gray')
plt.show()

In [ ]:
x_data = x_data[:,:,:,np.newaxis]
y_data = y_data[:,:,:,np.newaxis]
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size = 0.2)

In [ ]:
def my_generator(x_train, y_train, batch_size):
    data_generator = ImageDataGenerator(
            width_shift_range=0.1,
            height_shift_range=0.1,
            rotation_range=10,
            zoom_range=0.1).flow(x_train, x_train, batch_size, seed=SEED)
    
    mask_generator = ImageDataGenerator(
            width_shift_range=0.1,
            height_shift_range=0.1,
            rotation_range=10,
            zoom_range=0.1).flow(y_train, y_train, batch_size, seed=SEED)
    
    while True:
        x_batch, _ = data_generator.next()
        y_batch, _ = mask_generator.next()
        yield x_batch, y_batch

In [ ]:
image_batch, mask_batch = next(my_generator(x_train, y_train, 8))
fix, ax = plt.subplots(8,2, figsize=(8,20))
for i in range(8):
    ax[i,0].imshow(image_batch[i,:,:,0])
    ax[i,1].imshow(mask_batch[i,:,:,0])
plt.show()

DCUnet===========================
=======================================

In [ ]:


def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), activation='relu', name=None):
    '''
    2D Convolutional layers
    
    Arguments:
        x {keras layer} -- input layer 
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters
    
    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(1, 1)})
        activation {str} -- activation function (default: {'relu'})
        name {str} -- name of the layer (default: {None})
    
    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2D(filters, (num_row, num_col), strides=strides, padding=padding, use_bias=False)(x)
    x = BatchNormalization(axis=3, scale=False)(x)

    if(activation == None):
        return x

    x = Activation(activation, name=name)(x)

    return x


def trans_conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(2, 2), name=None):
    '''
    2D Transposed Convolutional layers
    
    Arguments:
        x {keras layer} -- input layer 
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters
    
    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(2, 2)})
        name {str} -- name of the layer (default: {None})
    
    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2DTranspose(filters, (num_row, num_col), strides=strides, padding=padding)(x)
    x = BatchNormalization(axis=3, scale=False)(x)
    
    return x


def DCBlock(U, inp, alpha = 1.67):
    '''
    DC Block
    
    Arguments:
        U {int} -- Number of filters in a corrsponding UNet stage
        inp {keras layer} -- input layer 
    
    Returns:
        [keras layer] -- [output layer]
    '''

    W = alpha * U

    #shortcut = inp

    #shortcut = conv2d_bn(shortcut, int(W*0.167) + int(W*0.333) 
    # + int(W*0.5), 1, 1, activation=None, padding='same')

    conv3x3_1 = conv2d_bn(inp, int(W*0.167), 3, 3, activation='relu', padding='same')

    conv5x5_1 = conv2d_bn(conv3x3_1, int(W*0.333), 3, 3,activation='relu', padding='same')

    conv7x7_1 = conv2d_bn(conv5x5_1, int(W*0.5), 3, 3,activation='relu', padding='same')

    out1 = concatenate([conv3x3_1, conv5x5_1, conv7x7_1], axis=3)
    out1 = BatchNormalization(axis=3)(out1)
    
    
    
    conv3x3_2 = conv2d_bn(inp, int(W*0.167), 3, 3, activation='relu', padding='same')

    conv5x5_2 = conv2d_bn(conv3x3_2, int(W*0.333), 3, 3, activation='relu', padding='same')

    conv7x7_2 = conv2d_bn(conv5x5_2, int(W*0.5), 3, 3, activation='relu', padding='same')
    
    out2 = concatenate([conv3x3_2, conv5x5_2, conv7x7_2], axis=3)
    out2 = BatchNormalization(axis=3)(out2) 
    
    
    conv3x3_3 = conv2d_bn(inp, int(W*0.167), 3, 3, activation='relu', padding='same')

    conv5x5_3 = conv2d_bn(conv3x3_3, int(W*0.333), 3, 3, activation='relu', padding='same')

    conv7x7_3 = conv2d_bn(conv5x5_3, int(W*0.5), 3, 3, activation='relu', padding='same')
    
    out3 = concatenate([conv3x3_3, conv5x5_3, conv7x7_3], axis=3)
    out3 = BatchNormalization(axis=3)(out3)
    

    out = add([out1, out2, out3])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    return out

def ResPath(filters, length, inp):
    '''
    ResPath
    
    Arguments:
        filters {int} -- [description]
        length {int} -- length of ResPath
        inp {keras layer} -- input layer 
    
    Returns:
        [keras layer] -- [output layer]
    '''

    shortcut = inp
    shortcut = conv2d_bn(shortcut, filters, 1, 1,activation=None, padding='same')

    out = conv2d_bn(inp, filters, 3, 3, activation='relu', padding='same')

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    for i in range(length-1):

        shortcut = out
        shortcut = conv2d_bn(shortcut, filters, 1, 1, activation=None, padding='same')

        out = conv2d_bn(out, filters, 3, 3, activation='relu', padding='same')

        out = add([shortcut, out])
        out = Activation('relu')(out)
        out = BatchNormalization(axis=3)(out)

    return out




def DCUNet(height, width, channels):
    '''
    DC-UNet
    
    Arguments:
        height {int} -- height of image 
        width {int} -- width of image 
        n_channels {int} -- number of channels in image
    
    Returns:
        [keras model] -- MultiResUNet model
    '''

    inputs = Input((height, width, channels))

    dcblock1 = DCBlock(32, inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(dcblock1)
    dcblock1 = ResPath(32, 4, dcblock1)

    dcblock2 = DCBlock(32*2, pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(dcblock2)
    dcblock2 = ResPath(32*2, 3, dcblock2)

    dcblock3 = DCBlock(32*4, pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(dcblock3)
    dcblock3 = ResPath(32*4, 2, dcblock3)

    dcblock4 = DCBlock(32*8, pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(dcblock4)
    dcblock4 = ResPath(32*8, 1, dcblock4)

    dcblock5 = DCBlock(32*8, pool4)
    pool5 = MaxPooling2D(pool_size=(2, 2))(dcblock5)
    dcblock5 = ResPath(32*8, 1, dcblock5)
    
    dcblock6 = DCBlock(32*16, pool5)

    up6 = concatenate([Conv2DTranspose(
        32*8, (2, 2), strides=(2, 2), padding='same')(dcblock5), dcblock4], axis=3)
    dcblock6 = DCBlock(32*8, up6)

    up7 = concatenate([Conv2DTranspose(
        32*4, (2, 2), strides=(2, 2), padding='same')(dcblock6), dcblock3], axis=3)
    dcblock7 = DCBlock(32*4, up7)

    up8 = concatenate([Conv2DTranspose(
        32*2, (2, 2), strides=(2, 2), padding='same')(dcblock7), dcblock2], axis=3)
    dcblock8 = DCBlock(32*2, up8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(
        2, 2), padding='same')(dcblock8), dcblock1], axis=3)
    dcblock9 = DCBlock(32, up9)

    conv10 = conv2d_bn(dcblock9, 1, 1, 1, activation='sigmoid')
    
    model = Model(inputs=[inputs], outputs=[conv10])
    
    return model

==============================================================================

In [ ]:
#model = MultiResUnet(height=IMG_HEIGHT , width=IMG_WIDTH, n_channels=1)

model = DCUNet(height=IMG_HEIGHT , width=IMG_WIDTH, channels=1)

=====================================================================================

In [ ]:
from keras.utils.vis_utils import plot_model
# Descripcion del modelo
#plot_model(model, show_shapes=True)


In [ ]:


def saveModel(model):

    model_json = model.to_json()

    try:
        os.makedirs('models')
    except:
        pass
    
    fp = open('models/modelP.json','w')
    fp.write(model_json)
    model.save('models/modelW.h5')



#model.summary()
#saveModel(model)

In [ ]:
from keras import backend as K



"""
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


"""


# different loss functions
def dice_coef(y_true, y_pred):
    smooth = 1.0  #0.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def jacard(y_true, y_pred):

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum ( y_true_f * y_pred_f)
    union = K.sum ( y_true_f + y_pred_f - y_true_f * y_pred_f)

    return intersection/union

def dice_coef_loss(y_true,y_pred):
    return 1 - dice_coef(y_true,y_pred)

def iou_loss(y_true,y_pred):
    return 1 - jacard(y_true, y_pred)

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.75
    smooth = 1
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)
def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)
def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)




def precision_xue(y_true, y_pred):
    """Precision metric.
   Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
   """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall_xue(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
"""


from keras import backend as K

def recall_m(y_true, y_pred):
    TP = tf.keras.metrics.TruePositives()
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = TP / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision



"""

In [ ]:
from keras.optimizers import Adam
from sklearn.metrics import auc

#initial_learning_rate=1e-3
opt = Adam(lr=initial_learning_rate)

#model.load_weights('./weights-unet.h5')

model.compile(optimizer=opt, #loss='mse',
              loss='binary_crossentropy',
              #loss=focal_tversky, 
              metrics=[keras.metrics.Precision(), 
                       keras.metrics.Recall(), 
                       #keras.metrics.SpecificityAtSensitivity(0.5), 
                       #keras.metrics.SensitivityAtSpecificity(0.5),
                       tf.keras.metrics.AUC(),
                       #f1_score,
                       'accuracy',          
              ])


In [ ]:

from keras.callbacks import TensorBoard, ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

#output_weights_name='weights.h5'
filepath = '../working/weights-unet.h5'
#TODO - VALIDATE THE LOGS OUTPUT
logs_base_dir = '../working/'
#patience_reduce_lr=2
#min_lr=1e-8
#filepath = '../working/'



reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,mode='min',
                              patience=patience_reduce_lr, min_lr=min_lr, verbose=1)

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_weights_only=True,
                                                 save_best_only=True, mode='max', period=1)
callbacks_conf = [checkpoint,
             TensorBoard(log_dir=os.path.join(logs_base_dir, "logs"),
                         batch_size=batchSize),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=early_stop_patience),
             #tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1),
             reduce_lr
             
            ]

"""



reduce_lr = ReduceLROnPlateau(monitor='val_dice_coef_loss', factor=0.1,mode='min',
                              patience=patience_reduce_lr, min_lr=min_lr, verbose=1)

checkpoint = ModelCheckpoint(filepath, monitor='val_dice_coef', verbose=1, save_weights_only=True,
                                                 save_best_only=True, mode='max', period=1)
callbacks_conf = [checkpoint,
             TensorBoard(log_dir=os.path.join(logs_base_dir, "logs"),
                         batch_size=batchSize),
             tf.keras.callbacks.EarlyStopping(monitor='val_dice_coef_loss', patience=early_stop_patience),
             #tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1),
             reduce_lr
             
            ]

"""

In [ ]:



import time
start = time.time()



fit_history = model.fit(x=x_train, y=y_train, 
                        validation_data = (x_val, y_val),
                        steps_per_epoch = steps_per_epoch,
                        epochs = epochs,                       
                        verbose = 1,
                        callbacks=callbacks_conf,
                       )



#Calculate execution time
end = time.time()
dur = end - start
if dur<60:
    print("Execution Time:",dur,"seconds")
elif dur>60 and dur<3600:
    dur=dur/60
    print("Execution Time:",dur,"minutes")
else:
    dur=dur/(60*60)
    print("Execution Time:",dur,"hours")

Epoch 00014: val_accuracy did not improve from 0.99154
Epoch 15/20
100/100 [==============================] - 2s 16ms/step - loss: 0.2414 - precision_1: 0.9934 - recall_1: 0.9843 - auc_1: 0.9993 - accuracy: 0.9947 - val_loss: 0.2367 - val_precision_1: 0.9899 - val_recall_1: 0.9728 - val_auc_1: 0.9948 - val_accuracy: 0.9917

Epoch 00015: val_accuracy improved from 0.99154 to 0.99170, saving model to ../working/weights-unet.h5
Execution Time: 2.1120710531870523 minutes

In [ ]:
model.load_weights('../working/weights-unet.h5')

In [ ]:
import matplotlib.pyplot as plt

"""
plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['dice_coef'], color='b')
plt.plot(hist.history['val_dice_coef'], color='r')
plt.show()
"""



plt.plot(fit_history.history["accuracy"])
plt.plot(fit_history.history["val_accuracy"])
#plt.plot(fit_history.history["val_auc_13"])
#plt.plot(fit_history.history["val_dice_coef"])

plt.plot(fit_history.history["loss"])
plt.plot(fit_history.history["val_loss"])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy", "loss","Validation Loss"])
plt.show()
plt.savefig('train_acc_val_plot.png')

In [ ]:
#plt.imshow(model.predict(x_train[0].reshape(1,IMG_HEIGHT, IMG_WIDTH, 1))[0,:,:,0], cmap='gray')

#plt.imshow(model.predict(x_val[0].reshape(1,IMG_HEIGHT, IMG_WIDTH, 1))[0,:,:,0], cmap='gray')

In [ ]:


# Evaluation Functions

def saveModel(model):

    model_json = model.to_json()

    try:
        os.makedirs('models')
    except:
        pass
    
    fp = open('models/modelP.json','w')
    fp.write(model_json)
    model.save('models/modelW.h5')


def evaluateModel(model, x_val, y_val, batchSize):
    
    try:
        os.makedirs('results')
    except:
        pass 

    
    yp = model.predict(x=x_val, batch_size=batchSize, verbose=1)
    yp = np.round(yp,0)
    
    for i in range(10):

        plt.figure(figsize=(20,10))
        plt.subplot(1,3,1)
        plt.imshow(x_val[i])
        plt.title('Input')
        plt.subplot(1,3,2)
        plt.imshow(y_val[i].reshape(y_val[i].shape[0],y_val[i].shape[1]))
        plt.title('Ground Truth')
        plt.subplot(1,3,3)
        plt.imshow(yp[i].reshape(yp[i].shape[0],yp[i].shape[1]))
        plt.title('Prediction')

        intersection = yp[i].ravel() * y_val[i].ravel()
        union = yp[i].ravel() + y_val[i].ravel() - intersection

        jacard = (np.sum(intersection)/np.sum(union))  
        plt.suptitle('Jacard Index'+ str(np.sum(intersection)) +'/'+ str(np.sum(union)) +'='+str(jacard))

        plt.savefig('results/'+str(i)+'.png',format='png')
        plt.close()
    
    jacard = 0
    dice = 0
    
    
    for i in range(len(y_val)):
        yp_2 = yp[i].ravel()
        y2 = y_val[i].ravel()
        
        intersection = yp_2 * y2
        union = yp_2 + y2 - intersection

        jacard += (np.sum(intersection)/np.sum(union))  

        dice += (2. * np.sum(intersection) ) / (np.sum(yp_2) + np.sum(y2))
        
        

    
    jacard /= len(y_val)
    dice /= len(y_val)
    


    print('Jacard Index : '+str(jacard))
    print('Dice Coefficient : '+str(dice))
    

    fp = open('models/log.txt','a')
    fp.write(str(jacard)+'\n')
    fp.close()

    fp = open('models/best.txt','r')
    best = fp.read()
    fp.close()
    

    if(jacard>float(best)):
        print('***********************************************')
        #print('Jacard Index improved from '+str(best)+' to '+str(jacard))
        print('***********************************************')
        fp = open('models/best.txt','w')
        fp.write(str(jacard))
        fp.close()
        
        saveModel(model)





In [ ]:

try:
    os.makedirs('models')
except:
    pass
    
fp = open('models/log.txt','w')
fp.close()
fp = open('models/best.txt','w')
fp.write('-1.0')
fp.close()

In [ ]:
evaluateModel(model,x_val, y_val,batchSize)

14/14 [==============================] - 2s 18ms/step
Jacard Index : 0.9669314987129636
Dice Coefficient : 0.9816574248495278
***********************************************
***********************************************

In [ ]:
# Show Predicted Samples


i=0 #From
z=10 #To

yp = model.predict(x=x_val, batch_size=batchSize, verbose=1)
yp = np.round(yp,0)

for i in range(z):

        plt.figure(figsize=(20,10))
        plt.subplot(1,3,1)
        plt.imshow(x_val[i])
        plt.title('Input')
        plt.subplot(1,3,2)
        plt.imshow(y_val[i].reshape(y_val[i].shape[0],y_val[i].shape[1]))
        plt.title('Ground Truth')
        plt.subplot(1,3,3)
        plt.imshow(yp[i].reshape(yp[i].shape[0],yp[i].shape[1]))
        plt.title('Prediction')